# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is classification problem. It's because we have exactly 2 groups of students: bad students, that could fail and other students.The output data is discrete, it's impossible to have a 0.5 failed student.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no'].shape[0]

# TODO: Calculate graduation rate
grad_rate = 100 * (n_passed) / (n_passed + n_failed) 

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.00%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()
print X_all[:1].to_string()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
from sklearn.cross_validation import train_test_split

num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
First, I desided to find strengths and weaknesses of all available models:

 * GaussianNB

Strengths: Easy to implement, works great with many features

Weaknesses: Could give bad result for continious features (but this is OK, because we don't have them)

Applied to:
Spam filtering. https://en.wikipedia.org/wiki/Naive_Bayes_spam_filtering
Object tracking in videos. http://ijcsma.com/publications/january2014/V2I108.pdf
Computer vision. http://research.google.com/pubs/pub41321.html
Medicine. http://scialert.net/fulltext/?doi=itj.2012.1166.1174&org=11

Can be applied to:
https://www.amazon.com/ to detect main actors closes or gadgets in films, that user likes(Object tracking in videos). Then add and link, were you can buy it))))))

 * Decision Trees

Strengths:  Easy to implement, tree could be converted to human readable scheme or picture.

Weaknesses: Overfit easily, sensitive to noise, problems with NaN and null data, not very good with continuous attributes

Applied to:
Financial analysis. It’s because minimum data preparation needed. And it is easy to explain results to business people what is going on, if make decision tree picture.
Medicine and Diagnostic.

 * Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)

Strengths: Allow to combine different estimators. If there are two of you, who work on improving prediction quality you can improve both of them, by making one ensemble Algorithm.

Weaknesses:You should know what method to choose(bagging or boosting) for particular data. Bagging is good, when model suffers from high variance. Boosting is good, when model suffers from high bias. But if you use Boosting for overfitting model, this will not improve improve prediction quality.

Applied to:
Optical Character Recognition. https://en.wikipedia.org/wiki/Optical_character_recognition
Medicine. In classification of cancer microarrays. http://link.springer.com/article/10.1007/s13721-013-0034-x
Kaggle competitions. Netflix price 2009 was won with gradient boosted decision trees and new XCBoost algorithm successfully used in many competitions in different domains.

Can be applied to:
https://www.airbnb.com to detect frauds and suspicious behaviour of host.

 * K-Nearest Neighbors (KNeighbors)

Strengths: Lazy evaluated. You don’t have to spend time, tuning algorithm

Weaknesses: You had to store all the data to make prediction. The more data you have, the slower is prediction. 
If there are many different classes around (very close to point that needs to be predicted) it is hard to make a prediction

Applied to:
Face detection in OpenCV.
Anomaly detection in data.

 * Stochastic Gradient Descent (SGDC)

Strengths: Efficiency. Allow to use many features (more than 10^5 features)

Weaknesses:  Is sensitive to feature scaling. Input data should be processed to make all attributes be on same scale (for example [-1; 1]). Many parameters to tune model!
(example from scikit-learn)
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

Applied to:
Text classification.
Natural language processing

 * Support Vector Machines (SVM)

Strengths: Effective in high dimensional spaces, Memory efficient, because use only subset of training points. Custom kernels could be specified.

Weaknesses: Give poor performance if number of features is much greater than the number of samples. (not in this case)
Do not provide probability estimates. Probabilities could be calculated after with expensive method. (there is no need in calculating probability right now). Doesn't perform well in large datasets. (our data set is limited). Doesn't perform good, if there are a lot of noise

Applied to:
Recognition of hand-written characters
In biology to classify proteins
Image segmentation

Can be applied to:
https://www.instagram.com/ To detect and hide obscene words on photo from children.(Image segmentation and Recognition of hand-written characters)


 * Logistic Regression

Strengths: Fast. Variables don’t have to be normal distributed. May handle nonlinear effects

Weaknesses: I requires many data to achieve good results. Overfitting

Applied to:
Medicine:  to predict mortality in injured patients, to predict whether a patient has diabetes or
coronary heart disease
Social science: to predict winner in elections.
Marketing: predict probability, that user will buy something 
Also in economics, business and engineering.


** Choice **

I think, that following features should be true for students intervention classificator:
It should not be sensitive to outliners (some child could be genious) 
and noise (people who track this data could make a mistake)
possibility to manage many features fast. (31)
And it would be great if result could be presented be some decision tree or scheme to explain result to teachers.

1) GaussianNB. Because it is fast, efficient and easy. It's disadvantages I found minor for this data.

2) Random forest. As ensemble method it should be fast too. And it return feature_importances_ that could be used to make presentation for teachers how to prevent students from failure

3) SVM. as GaussianNB it should be fast and efficient too, because we no need a probabilities right now.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

clf_A = GaussianNB()
clf_B = RandomForestClassifier()
clf_C = SVC()
clfs = [clf_A, clf_B, clf_C]

def splitDataInto(train_size):
    return (X_all[:train_size], X_all[train_size:], y_all[:train_size], y_all[train_size:])

splits = [splitDataInto(100), splitDataInto(200), splitDataInto(300)]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

for clf in clfs:
    for split in splits:
        X_train, X_test, y_train, y_test = split
        train_predict(clf, X_train, y_train, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0029 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 0.4536.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.2692.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0009 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8071.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.6667.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0015 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8246.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6435.
Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0296 seconds
Made predictions in 0.0015 seconds.
F1 score for training set: 0.9867.
Made predictions in 0.0016 seconds.
F1 score for test set: 0.7106.
Training a RandomForestClassifier using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0014         |         0.0004         |       0.4536     |      0.2692     |
| 200               |          0.0008         |         0.0004         |       0.8071     |      0.6667     |
| 300               |          0.0009         |         0.0003         |       0.8246     |      0.6435     |

** Classifer 2 - RandomForestClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0381             |        0.0014          |      0.9933      |      0.7105     |
| 200               |      0.0328             |        0.0013          |      0.9853      |      0.6531     |
| 300               |      0.0311             |        0.0012          |      0.9876      |      0.5472     |

** Classifer 3 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0013            |          0.0015        |      0.8772      |      0.7792     |
| 200               |       0.0032            |          0.0028        |      0.8474      |      0.7987     |
| 300               |       0.0062            |          0.0017        |      0.8686      |      0.7571     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** The best result (fast training time and high F1 score) shows SVM Classificator. So, it should be the best model.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** There are very successfull students, who obviosly will pass all of the exams. There are such a bad students, that definitely fail all of the exams. You don't need a fancy algorithm to define that. The most interesting case is where it is unknown if the student pass or fail. The main purpose of this model is to find the most accurate boundary that separate cases where students most probably fail, from cases where students most probably pass. So, when algorithm is working he tries to mark this edge cases as more important, then obvious cases. After that, algorithm try to define such a border between "pass" and "fail" students. Imagine one student, that should fail, but passed and student, that almost passed, but failed. Algorithm analyze difference between them, and try to find such an conditions, where its imposible to say, if the student fail or pass. Then, algorithm use this conditions to define if any other student pass or fail.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

parameters = { 'kernel' : ['rbf', 'poly', 'linear', 'sigmoid'], 'C': [1.0, 0.01, 10], 'tol': [0.001, 0.01, 0.005], 'gamma': [0.1, 0.01, 0.001] }

clf = SVC()

f1_scorer = make_scorer(f1_score, pos_label="yes")

grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0042 seconds.
Tuned model has a training F1 score of 0.8301.
Made predictions in 0.0013 seconds.
Tuned model has a testing F1 score of 0.7832.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** F1 score(train) = 0.8301, F1 score(test) = 0.7832. Result of tuned model is better than F1 score(test) = 0.7571 for the same amount test/train examples.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.